In [1]:
import pandas as pd #the data will be saved locally as a csv file. Pandas is a nice way to write/read/work with those files.
import numpy as np #to do math
import matplotlib.pyplot as plt #To make plots
%matplotlib inline
import glob #to import file names of TMS snapshots recorded as csv files, to be later combined in dataframe
from datetime import timedelta #for accurate plotting of dates%matplotlib inline
from datetime import datetime

In [2]:
filenames=glob.glob('listingsSnapShot*.csv')
filenames=sorted(filenames,reverse=True)

In [3]:
filenames[0]

'listingsSnapShot170508.csv'

In [4]:
df=pd.read_csv(filenames[0],encoding='ISO-8859-1')

In [5]:
for x in range(1,len(filenames)):
    dftemp=pd.read_csv(filenames[x],encoding='ISO-8859-1');
    df_listing=df.loc[:,'listing'].values;
    dftemp_listing=dftemp.loc[:,'listing'].values;
    unique_listings=[num for num in dftemp_listing if num not in df_listing];
    dftemp2=dftemp.loc[dftemp.loc[:,'listing'].isin(unique_listings),:].copy();
    #these next lines account for postings being pulled before the closing date
    #non-ideal fix: when program notices posting is gone before it should be,
    #change closing date to date of recording where posting first noticed to be missing
    scrape_date=datetime.strptime(filenames[x-1].split('.')[0][-6:],'%y%m%d')
    dftemp2.loc[pd.to_datetime(dftemp2.loc[1:,'Closing Date'])>scrape_date,
            'Closing Date']=scrape_date.strftime('%m/%d/%Y').lstrip('0')
    frames = [df, dftemp2]
    result = pd.concat(frames)
    result.reset_index(inplace=True)
    del result['index']
    df=result

In [6]:
len(df)

364

In [7]:
df2=pd.DataFrame(columns=['Listings','HTML','Position Title', 'Agency/Sub-Agency',
       'Opening Date', 'Closing Date', 'Number of Positions','State', 'Series', 'GS-09','GS-11','GS-12','GS-13',
       'Promotion Potential'])
x=0
y=0
for x in range(0,len(df)):
    subtest1=[]
    subtest2=[]
    z=0
    c=df.loc[x,'Positions'].split();
    for elm in c:
        if elm=='in':
            try:
                subtest1.append(int(c[z-1]))
                #print(int(c[x-1]))
            except:
                subtest1.append(int(c[z-1][2:]))
                subtest2.append(c[z-1][0:2])
        z=z+1
    subtest2.append(c[-1])
    for w in range(0,len(subtest1)):
        df2.loc[y,'Listings']=df.loc[x,'listing']
        df2.loc[y,'HTML']=df.loc[x,'info']
        df2.loc[y,'Position Title']=df.loc[x,'Position Title']
        df2.loc[y,'Agency/Sub-Agency']=df.loc[x,'Agency/Sub-Agency']
        df2.loc[y,'Opening Date']=df.loc[x,'Opening Date']
        df2.loc[y,'Closing Date']=df.loc[x,'Closing Date']
        df2.loc[y,'Series']=df.loc[x,'Series'][0:4]
        df2.loc[y,'Position Title']=df.loc[x,'Position Title']
        df2.loc[y,'Position Title']=df.loc[x,'Position Title']
        df2.loc[y,'Position Title']=df.loc[x,'Position Title']
        df2.loc[y,'GS-09']= 'GS-09' in df.loc[x,'Grade Level']
        df2.loc[y,'GS-11']= 'GS-11' in df.loc[x,'Grade Level']
        df2.loc[y,'GS-12']='GS-12' in df.loc[x,'Grade Level']
        df2.loc[y,'GS-13']='GS-13' in df.loc[x,'Grade Level']
        df2.loc[y,'Promotion Potential']=df.loc[x,'Promotion Potential']
        df2.loc[y,'Number of Positions']=subtest1[w]
        #df2.loc[y,'State']=subtest2[w]
        df2.loc[y,'State']=subtest2[1][0:2]
        y=y+1

In [8]:
dfearly=pd.read_csv('meta_early.csv',encoding='ISO-8859-1')

In [9]:
dfearly.head(2)

,Listings,Position Title,Agency/Sub-Agency,Opening Date,Closing Date,Number of Positions,State,Series,GS-09,GS-11,GS-12,GS-13,Promotion Potential
0,3,Research Analyst,Broadcasting Board of Governors,2017-01-13 00:00:00,2017-12-13 00:00:00,1,DC,343,False,True,False,False,GS-13
1,9,Management and Program Analyst,Department of the Treasury / Bureau of the Fis...,2017-01-18 00:00:00,2017-03-01 00:00:00,1,DC,343,False,False,False,False,GS-12


In [10]:
df2_listing=df2.loc[:,'Listings'].values;
dfearly_listing=dfearly.loc[:,'Listings'].values;
unique_listings=[num for num in dfearly_listing if num not in df2_listing];
dftemp2=dfearly.loc[dfearly.loc[:,'Listings'].isin(unique_listings),:].copy();
scrape_date=datetime.strptime(filenames[-1].split('.')[0][-6:],'%y%m%d')
dftemp2.loc[pd.to_datetime(dftemp2.loc[1:,'Closing Date'])>scrape_date,
        'Closing Date']=scrape_date.strftime('%m/%d/%Y').lstrip('0')
frames = [df2, dftemp2]
result = pd.concat(frames)
result.reset_index(inplace=True)
del result['index']
df2=result

In [11]:
len(df2)

435

In [12]:
#number of postings without html attached
len(df2[df2.loc[:,'HTML'].isnull()])

23

In [13]:
df2=df2.sort_values('Listings').copy()
df2.reset_index(inplace=True)
del df2['index']

In [14]:
df2.to_csv('meta_composite2.csv',index=False)